《深入理解 Tensorflow 架构设计与实现原理》第4章， Tensorflow 数据处理方法
- 输入数据集
- 模型参数
- 命令行参数

# 输入数据集

###  使用Tensorflow input pipeline 读取
- 创建文件名列表  
- 创建文件名队列  
- 创建 Reader 和 Decoder  
- 创建样例队列  

![](https://upload-images.jianshu.io/upload_images/4685306-eae218123ff6abd2.gif?imageMogr2/auto-orient/strip%7CimageView2/2/w/700)
#### 创建文件名列表
- 使用 python list　　
- 使用 ```tf.train.match_filenames_once``` 返回一个存储了符合该匹配模式的文件名列表变量  
```python
def match_filenames_once(pattern, name=None):
  """Save the list of files matching pattern, so it is only computed once.
  NOTE: The order of the files returned can be non-deterministic.
  Args:
    pattern: A file pattern (glob), or 1D tensor of file patterns.
    name: A name for the operations (optional).
  Returns:
    A variable that is initialized to the list of files matching the pattern(s).
```

#### 创建文件名队列
- 使用 ```tf.train.string_input_producer()``` 遍历整个数据集，输入是前面创建的文件名列表，因此其参数 num_epochs 表示训练的epoch数

```python
def string_input_producer(string_tensor,
                          num_epochs=None,
                          shuffle=True,
                          seed=None,
                          capacity=32,
                          shared_name=None,
                          name=None,
                          cancel_op=None):
    """
    - string_tensor 存储文件名列表的字符串张量， A 1-D string tensor with the strings to produce
    - num_epochs 训练周期
    - shufftle 是否打乱文件名顺序
    - capacity 文件名队列容量，在图像处理中用的会比较多吧，毕竟图像是一张一张的，而文本基本上都是存储在一个csv文件中
    """
```

#### 创建 Reader 和 Decoder
|文件格式|Reader类型| Decoder类型|  
|---|---|---|
|CSV文件|tf.TextLineReader|tf.decoder_csv|
|TFRecord文件|tf.TFRecordReader|tf.parse_single_example|
|自由格式|tf.FixedLengthRecordReader|tf.decoder_raw|

##### CSV 文件
纯文本形式存储，多条记录间以换行符分隔，每条记录由多个字段组成，字段间以制表符或逗号分隔。

使用 ```tf.TextLineReader``` 读取数据 https://www.tensorflow.org/api_docs/python/tf/TextLineReader

In [1]:
# 用 tf.TextLineReader 
# 这里我们数据集只有一个文件 train.csv 所以文件名列表用 python list 表示
import tensorflow as tf
# 创建文件名队列，因为只有一个文件，所以文件名列表就省了
filename_queue = tf.train.string_input_producer(["train.csv"])
# 创建读取 Reader
reader = tf.TextLineReader()
# 从文件名队列中读取 CVS 中的一条数据记录 value
_, value = reader.read(filename_queue)
# 设置数据记录的默认值，当记录中有缺省值时，用默认值填充
record_defaults = [[],[],[],[],[],[]]
file_id, label, api, tid, return_value, index = tf.decode_csv(value,record_defaults=record_defaults)
features = tf.stack([file_id, label, api, tid, return_value, index])

/home/panxie/anaconda3/envs/NLP/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/panxie/anaconda3/envs/NLP/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


除了可以用 tf.TextLineReader.read 一次读取一条记录之外，还可以用 tf.TextLineReader.read_up_to()

##### TFRecords 文件
存储的是有结构的序列化字符块，它是 Tensorflow 推荐的标准文件格式。

###### 使用 tf.python_io.TFRecordWriter 方法将数据保存为 TFRecord 文件

In [ ]:
'''writer.py'''
# 创建向 TFRecords 文件写数据记录的 writer
writer = tf.python_io.TFRecordWriter('stat.tfrecord')
# 循环构造输入样例
for i in range(1,3):
    # 创建 example.proto 中定义的样例
    example = tf.train.Example(
        features = tf.train.Features(
            feature = {
            "id":tf.train.Feature(int64_List = tf.train.Int64_List(value=[i])),
            ...
            }
        )
    )
    # 将样例序列化为字符串后，写入 stat.TFRecord 文件
    writer.write(example.SerializeToString())
# 关闭输出流
writer.close()

### 显然当文件很多的时候可以采用多个 stat.tfrecord 文件来存储

###### 使用 tf.TFRecordReader 方法读取 stat.TFRecord 文件中的样例，接着使用 tf.sparse_single_example 将样例转换为张量

In [ ]:
'''reader.py'''
# 创建文件名队列
filename_queue = tf.train.string_input_producer(['stat.tfrcord'])
# 创建读取 TFRecord 文件的 reader
reader = tf.TFRecordReader()
# 取出 stat.TFRecord 文件中的一条序列化的样例 serialized_example
_, serialize_sample = reader.read(filename_queue)  # 返回 key,value
# 将一条序列化的样例转换为其包含的所有特征张量
features = tf.sparse_single_example(
    serialize_sample,
    features = {
        "id":tf.FixedLenFeature([], tf.int64)
        ...
    }
)


##### 自由格式文件
用户自定义的二进制文件。它的存储对象是字符串，每条记录是一个固定长度的字节块。因此，首先使用 ```tf.FixedLengthRecordReader``` 读取二进制文件中固定长度的字节块，然后使用 ```tf.decode_raw``` 方法将读取的字符串转换为 unit8 类型的张量。

##### 创建样例队列
通过 Reader 和 Decoder 我们得到了特征张量 features，然后在会话执行时，我们需要使用 ```tf.train.start_queue_runners``` 方法启动入队操作的所有线程。

In [ ]:
# 接着 TFRecord 的reader 文件
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
# 启动执行入队操作的后台线程
tf.train.start_queue_runners(sess=sess)
# 读取第一条数据
for i in range(1,2):
    example = sess.run(features)
    print(example)

这种方法容错行很差，因为在队列操作后台线程的生命周期“无人管理”，任何线程出现错误，都会出现异常，导致程序崩溃。

常见的是文件名或样例队列越界抛出的 ```tf.errors.OutOfRangeError```. 队列越界通常的原因是读取数据记录的数量超过了 ```tf.train.string_input_producer``` 方法中 num_epochs 数据集便利次数。

为了处理这种异常，我们使用 ```tf.train.Coordinator``` 方法管理多线程生命周期的协调器。当某个线程出现异常时，它的 should_stop 成员返回 True， for循环结束。然后程序执行 finally 中协调器的 request_stop 成员方法，请求所有线程安全退出。具体 code 在之后例子中使用。

##### TFRecords 样例数据结构
样例和特征的层次结构。Features 由形如 <string, Feature> 的feature字典组成。其中 feature字典的值的数据结构为 Feature.

### 创建批样例数据的方法
前面的最后，我们得到了很多样例，但是这些样例要打包聚合成批数据才能提供模型训练、评估和推理使用。Tensorflow 提供的 ```tf.train.shuffle_batch``` 方法能够使用样例建立批数据，而且能够在打包过程中打乱样例顺序。
```python
tf.train.shuffle_batch(
    tensors,             # 列表或字典，The list or dictionary of tensors to enqueue.
    batch_size,   
    capacity,            # 样例队列中总的样例个数，最好设置为：min_after_queue + (num_threads + margin)*batch_size
    min_after_dequeue,   # 样例队列中出队的样例个数
    num_threads=1,
    seed=None,
    enqueue_many=False,
    shapes=None,
    allow_smaller_final_batch=False,
    shared_name=None,
    name=None
)
```

具体参数设置参考：https://github.com/tensorflow/tensorflow/blob/25c197e02393bd44f50079945409009dd4d434f8/tensorflow/python/training/input.py#L1204

以及后续的使用中去进一步了解。

In [7]:
### 这里是完整输入流水线的伪代码
def get_my_example(filename_queue):
    reader = tf.SomeRead()            # csv 使用 tf.TextLineReader, TFRecord 使用 tf.TFRecordReader 
    _, value = reader.read(filename_queue)
    feature = tf.decode_some(value)
    # 对样例进行预处理
    processed_example = some_processing(features)
    return processed_example

def input_pipeline(filename, batch_size, num_epoches=None):
    # 当 num_epochs == None时表示文件队列总是可用的，一直循环入队
    filename_queue = tf.train.string_input_producer(
        filenames, nochs=num_epochs, shuffle=True)  # 创建文件名队列，并shuffle
    example = getum_ep_my_example(filename_queue)   # 通过 Reader 和 Decoder 将文件转换为特征张量
    # min_after_dequeue 表示从样例队列中出队的样例个数,值越大表示打乱的越好
    min_after_dequeue = 1000
    # capacity 表示批数据队列的容量，推荐设置
    # capacity = min_after_dequeue + (num_threads + a small safety margin) * batch_size
    capacity = min_after_dequeue + 3 * batch_size
    # 创建批样例 example_batch
    example_batch = tf.train.shuffle_batch(
        [example], batch_size=batch_size, capacity=capacity,
        min_after_dequeue = min_after_dequeue)
    return example_batch   

### 填充数据节点的方法
不需要存储完整数据集，有效减小了内存开销．使用流水线也保证了实时性．

In [ ]:
# 使用上面定义的 input_pipeline 方法获取批样例 x_batch
x_batch = input_pipeline(['stat.tfrecord'], batch_size=64)
# 模型
sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(),
                  tf.local_variables_initializer())   # 在使用协调器管理多线程之前，先对其初始化
sess.run(init_op)
# 创建协调器
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
try:
    for _ in range(1000):
        if not coord.should_stop():
            sess.run(train_op)
            print(example)
except tf.error.OutOfRangeError:
    print("Catch OutOfRangeError")
finally:
    #　请求停止后台线程
    coord.request_stop()
    print("Finish reading")
#　等待所有后台线程安全退出
coord.join(threads)
sess.close()

## 动手实践
以数据集 cifar-10 为例，采用 pipeline 输入。
[cifar-input.py]()

这一部分比较难理解，参考知乎上的文章，在重新理解一遍:https://zhuanlan.zhihu.com/p/27238630

tensorflow 读取数据，主要有两个队列。分别是文件名队列和样本队列。
![](https://pic3.zhimg.com/80/v2-3cd597df7e855af6d59ff60af6b13cb2_hd.jpg)

### 文件名队列
首先是文件名队列，```tf.train.string_input_producer```.
```python
tf.train.string_input_producer(
    string_tensor,
    num_epochs=None,
    shuffle=True,
    seed=None,
    capacity=32,
    shared_name=None,
    name=None,
    cancel_op=None
)
```

参数说明：  
- string_tensor: A 1-D string tensor with the strings to produce. 文件名字符串，也可以是文件名字符串的list
- num_epochs：integer，如果有定义，那就遍历整个数据集 num_epochs次，如果没有定义，那就无限循环。
- shuffle: 是否打乱
- capacity: 队列容量

In [1]:
!ls dataset

batches.meta  data_batch_2  data_batch_4  readme.html
data_batch_1  data_batch_3  data_batch_5  test_batch


In [3]:
import tensorflow as tf
import os

# 文件名列表
filename_list = [os.path.join("dataset", "data_batch_{}".format(i)) for i in range(1,6)]

In [4]:
# 创建文件名队列
file_queue = tf.train.string_input_producer(filename_list, num_epochs=10, shuffle=True)
file_queue

除了```tf.train.string_input_producer```外，我们还要额外介绍一个函数：```tf.train.start_queue_runners```。初学者会经常在代码中看到这个函数，但往往很难理解它的用处，在这里，有了上面的铺垫后，我们就可以解释这个函数的作用了。

在我们使用tf.train.string_input_producer创建文件名队列后，整个系统其实还是处于“停滞状态”的，也就是说，我们文件名并没有真正被加入到队列中（如下图所示）。此时如果我们开始计算，因为内存队列中什么也没有，计算单元就会一直等待，导致整个系统被阻塞。

### Reader and Decoder
这里是二进制文件，可以采用 ```tf.FixedLengthRecordReader```函数来读取

In [5]:
# 标签为1字节
LABEL_BYTES = 1
# 图像大小
IMAGE_SIZE = 32
# 通道数
IMAGE_DEPTH = 3
# 图片数据为 32*32*3=3072 字节
IMAGE_BYTES = IMAGE_SIZE * IMAGE_SIZE * IMAGE_DEPTH
# 标签数
NUM_CLASSES = 10

In [6]:
record_bytes = IMAGE_BYTES + LABEL_BYTES
reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)

In [7]:
key,value = reader.read(file_queue)
key, value

(<tf.Tensor 'ReaderReadV2:0' shape=() dtype=string>,
 <tf.Tensor 'ReaderReadV2:1' shape=() dtype=string>)

可以看到读取出来的是 string 的形式。我们需要的 label 和 图像像素保存在 value 中。然后使用 decode_raw 将 value 转换成图片像素的 uint8 形式。

In [9]:
# 将字节信息转换成张量
record = tf.reshape(tf.decode_raw(value, out_type=tf.uint8), shape=[record_bytes])
record # 可以看出reader只读取了一张图片的信息

<tf.Tensor 'Reshape_1:0' shape=(3073,) dtype=uint8>

In [14]:
# tf.slice 的输入是 tensor
# 需要提前了解 cifar 中数据的组织形式，
label = tf.cast(tf.slice(record, begin=[0], size=[LABEL_BYTES]), tf.int32)
image_depth_major = tf.reshape(tf.slice(record, [1], [IMAGE_BYTES]), [IMAGE_DEPTH, IMAGE_SIZE, IMAGE_SIZE])
image_depth_major

<tf.Tensor 'Reshape_4:0' shape=(3, 32, 32) dtype=uint8>

In [17]:
image = tf.cast(tf.transpose(image_depth_major,[1,2,0]), tf.float32)
image

<tf.Tensor 'Cast_4:0' shape=(32, 32, 3) dtype=float32>

### 批量读取数据
我们得到了很多样例，但是这些样例要打包聚合成批数据才能提供模型训练、评估和推理使用。Tensorflow 提供的 tf.train.shuffle_batch 方法能够使用样例建立批数据，而且能够在打包过程中打乱样例顺序。
```python
tf.train.shuffle_batch(
    tensors,             # 列表或字典，The list or dictionary of tensors to enqueue.
    batch_size,   
    capacity,            # 样例队列中总的样例个数，最好设置为：min_after_queue + (num_threads + margin)*batch_size
    min_after_dequeue,   # 样例队列中出队的样例个数
    num_threads=1,
    seed=None,
    enqueue_many=False,
    shapes=None,
    allow_smaller_final_batch=False,
    shared_name=None,
    name=None
)
```

In [18]:
# min_after_dequeue defines how big a buffer we will randomly sample
min_after_dequeue = 5000
num_threads = 8
batch_size = 64
capacity = min_after_dequeue + num_threads * batch_size

In [20]:
image_batch, label_batch = tf.train.shuffle_batch([image, label],
                                                batch_size=batch_size,
                                                capacity=capacity,
                                                num_threads=num_threads,
                                                min_after_dequeue=min_after_dequeue)

In [28]:
sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(),
                  tf.local_variables_initializer())   # 在使用协调器管理多线程之前，先对其初始化
sess.run(init_op)
# 启动执行入队操作的后台线程
tf.train.start_queue_runners(sess=sess)
print(sess.run(image_batch[0,0,0,:]))
print(sess.run(image_batch[0,0,0,:]))
print(sess.run(image_batch[0,0,0,:]))
print(sess.run(image_batch[0,0,0,:]))

[ 37.  70. 161.]
[120.  85.  39.]
[141. 149.  62.]
[172. 183. 195.]


整合到py文件中，作为模型的输入。

In [ ]:
sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(),
                  tf.local_variables_initializer())   # 在使用协调器管理多线程之前，先对其初始化
sess.run(init_op)
# 启动执行入队操作的后台线程
tf.train.start_queue_runners(sess=sess)
# 读取第一条数据
i = 0
while True:
    i += 1
    if i > 3:
        break
    example = sess.run(value)
    print(i, example)